In [4]:
from mysql.connector.pooling import MySQLConnectionPool
from mysql.connector import Error
import pandas as pd

In [ ]:
dbconfig = {
    "user": "user1",
    "password": "Arcep@1990",
    "host": "localhost",
    "database": "little_lemon_db"
}

# Create the connection pool
def create_pool(pool_size, pool_name): 
    try:
        pool = MySQLConnectionPool(pool_name = pool_name,
                            pool_size = pool_size,
                            **dbconfig)
        print("The connection pool is created with a name: ",pool.pool_name)
        print("The pool size is:",pool.pool_size)
        return pool

    except Error as er:
        print("Error code:", er.errno)
        print("Error message:", er.msg)

def query_db(sql_query='', pool=None, values=None, fetch=True): 
    try: 
        connection = pool.get_connection()
        cursor = connection.cursor()

        if values:  # Vérifie si on doit exécuter plusieurs insertions
            cursor.executemany(sql_query, values)
        else:
            cursor.execute(sql_query)

        result = None
        if fetch and cursor.with_rows:  # ✅ Vérifie si la requête retourne un résultat
            result = cursor.fetchall()
        
        while cursor.nextset():  # 🔥 Vide tous les résultats restants
            pass

        connection.commit()
        return result

    except Exception as e:
        print(f"Error occurred: {e}")
        if connection.is_connected():
            connection.rollback()

    finally:
        if 'cursor' in locals():
            cursor.close()
        if 'connection' in locals() and connection.is_connected():
            connection.close()

            

In [14]:
# Create pool (size 3)
first_pool = create_pool(3, "first_pool")


# Exécution de plusieurs insertions
query_db(sql_query=drop_query, pool=first_pool, fetch=False)
query_db(sql_query=create_query, pool=first_pool, fetch=False)
query_db(sql_query=insert_query, values=insert_values, pool=first_pool, fetch=False)




The connection pool is created with a name:  first_pool
The pool size is: 3
